In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
crunchbase=pd.read_csv('crunchbase_complete.csv')
crunchbase=crunchbase.set_index("Unnamed: 0")

In [6]:
industries_list=[x.split(",") for x in crunchbase.industry]

In [7]:
## fix the empties
for i in range(len(industries_list)):
        industries_list[i]=[x for x in industries_list[i] if len(x)>0]
        industries_list[i]=[str(industries_list[i][j])[1:]  
                            if str(industries_list[i][j])[0]==' ' else industries_list[i][j]
                           for j in range(len(industries_list[i]))
                           ]

In [8]:
year_ind={}
for year in range(2012, 2020):
    year_ind[year]=list(crunchbase[crunchbase.year==year].index)

In [57]:
keys=list(np.unique([x for y in industries_list for x in y]))

In [12]:
industries_values=pd.DataFrame(index=keys)

In [13]:
for year in range(2012, 2020):
    for key in industries_values.index:
        industries_values.loc[key, year]=np.sum([crunchbase['usd_value'][ind] 
                                       for ind in year_ind[year] if key in industries_list[ind]])/10**6

In [36]:
def growth_calc(industry):
    first_year=[x for x in industries_values.loc[industry] if x!=0][0]
    return np.float(industries_values.loc[industry][2019]/first_year)**(1/7)

In [37]:
industries_values['growth']=pd.Series([growth_calc(industry) for industry in industries_values.index], 
                                      index=industries_values.index)

In [62]:
top_industries=[]
for year in range(2012, 2020):
    top_industries.extend(list(industries_values.sort_values(year)[::-1].iloc[:50,:].index))
top_industries=list(np.unique(top_industries))

In [83]:
for x in list(np.arange(0,100,25)):
    print(x, np.percentile(list(industries_values.loc[top_industries]['growth'][industries_values.growth<1]), x)-1)

0 -0.452590053517
25 -0.171400834809
50 -0.0645641629331
75 -0.0408702364834


In [44]:
def get_companies(key, year):
    result=[]
    ind=[i for i in range(len(industries_list)) if key in industries_list[i]]
    companies=crunchbase.loc[ind].groupby(['year', 'company']).sum()['usd_value']/10**6
    companies=companies[year].sort_values()[::-1]
    companies=companies[companies>1]
    companies_dict=dict(companies.iloc[:20])
    
    for company in companies_dict.keys():
        investors=list(crunchbase.loc[ind][(crunchbase.company==company)&(crunchbase.year==year)]['lead_investors'])
        if '—' in investors:
            investors.remove('—')
        result.append({'company': company, 'value': companies_dict[company], 
                       'investors': ', '.join(investors),
                       'descr':list(crunchbase.loc[ind][crunchbase.company==company]['descrip'])[0]})
    return result

In [58]:
cosine_matrix={}
cosine_matrix_short={}
tf_idf = {}
nodes={}

In [ ]:
#old_keys_short=[]

for year in range(2012, 2020):
    keys_short=list(industries_values.sort_values(year, ascending=False).iloc[:50].index)
    
    keys_short.extend(new_keys_short)
    keys_short=list(np.unique(keys_short))
    
    order=list(industries_values.sort_values(year, ascending=False).index)
    keys_short=[x for x in keys_short if order.index(x)<100]
    
    tf_idf[year]=pd.read_csv(str('tf_idf_'+str(year)+'.csv')).iloc[:,1:]
    print(tf_idf[year].shape)
    
    x_cos=cosine_similarity(tf_idf[year], tf_idf[year])
    cosine_matrix[year]=pd.DataFrame(x_cos, columns=keys, index=keys)
        
    cos_sim_short=cosine_matrix[year].loc[keys_short, keys_short]
    cos_sim_short.to_csv(str('cos_sim_'+str(year)+'.csv'))
    
    cos_sim_short=pd.read_csv(str('cos_sim_'+str(year)+'.csv')).iloc[:,1:]
    nodes[year]={"nodes":[], "links":[]}

    for key in keys_short:
        try: 
            companies=get_companies(key, year)
        except:
            companies=[]
            
        nodes[year]['nodes'].append({'name':key, 'radius': int(np.sqrt(industries_values.loc[key, year])), 
                                     'growth': industries_values.loc[key, 'growth'],
                                    'companies': companies})        
        

    for i, key in enumerate(keys_short):
        for j, h_key in enumerate(keys_short[i:]):
            if (i!=j+i)&(cos_sim_short.loc[key, h_key]>=0.1):
                nodes[year]['links'].append({"source":i,"target":j+i,"value":cos_sim_short.loc[key, h_key]})            
    
    with open(str('nodes.json'), 'w') as outfile:
        json.dump(nodes, outfile)

In [22]:
industry_type={'Association':'E-Commerce', 'E-Commerce':'E-Commerce', 'Fashion': 'E-Commerce', 'Internet': 'E-Commerce',
               'Food Delivery':'E-Commerce', 'Delivery':'E-Commerce', 'Marketplace':'E-Commerce', 'Grocery':'E-Commerce',
               
              'Medical': 'Medical', 'Medical Device':'Medical', 'Medical Device':'Medical', 'Health Care': 'Medical', 'Biopharma': 'Medical',
               'Health Diagnostics':'Medical', 'Biotechnology':'Medical', 'Pharmaceutical':'Medical', 'Emergency Medicine':'Medical',
               
               'Information Technology':'IT', 'Apps':'IT', 
               'Software': 'IT', 'Enterprise Software': 'IT', 'Mobile Apps':'IT', 'Information Services':'IT', 
               
               'Finance':'Finance', 'Financial Services':'Finance', 'FinTech':'Finance', 'Banking':'Finance', 'Blockchain':'Finance',
               
              'Oil and Gas':'Energy & Natural Resources', 'Clean Energy':'Energy & Natural Resources','Energy':'Energy & Natural Resources',
               'Natural Resources':'Energy & Natural Resources', 
              'Renewable Energy':'Energy & Natural Resources', 'Mining':'Energy & Natural Resources', 'Precious Metals':'Energy & Natural Resources',
               
               'Public Transportation':'Transportation', 'Ride Sharing':'Transportation',
               'Logistics':'Transportation', 'Automotive':'Transportation', 'Transportation':'Transportation',
               'Car Sharing':'Transportation', 'Autonomous Vehicles':'Transportation',
              
               'Insurance':'Insurance', 'Auto Insurance': 'Insurance', 'Property Insurance': 'Insurance',
              
               'Analytics':'AI & Big Data', 'Big Data': 'AI & Big Data', 'Cloud Computing':'AI & Big Data', 'Cloud Data Services':'AI & Big Data',
              
               'Artificial Intelligence':'AI & Big Data', 'Database':'AI & Big Data', 'Data Storage':'AI & Big Data', 
               'Data Center':'AI & Big Data'}

In [42]:
def get_companies(key, year, stage):
    result=[]
    ind=[i for i in range(len(industries_list)) if (key in industries_list[i])&(crunchbase.funding_type[i] in stages[stage])]
    companies=crunchbase.loc[ind].groupby(['year', 'company']).sum()['usd_value']/10**6
    companies=companies[year].sort_values()[::-1]
    companies=companies[companies>1]
    companies_dict=dict(companies.iloc[:20])
    
    for company in companies_dict.keys():
        investors=list(crunchbase.loc[ind][(crunchbase.company==company)&(crunchbase.year==year)]['lead_investors'])
        if '—' in investors:
            investors.remove('—')
        result.append({'company': company, 'value': companies_dict[company], 
                       'investors': ', '.join(investors),
                       'descr':list(crunchbase.loc[ind][crunchbase.company==company]['descrip'])[0]})
    return result

In [43]:
def industry_group(k):
    try: return industry_type[k]
    except: return 'Other'

In [36]:
dataset=pd.DataFrame()

for year in range(2012, 2020):
    for s in range(len(stages)):
        dataset=pd.concat([dataset, crunchbase_group.loc[year].loc[s].sort_values('usd_value')[::-1].iloc[:15]])
dataset['group']=pd.Series([industry_group(key) for key in dataset.index], index=dataset.index)
order=list(dataset.groupby('group').sum()['usd_value'].sort_values(ascending=False).index)

In [41]:
top_industries={}
for year in range(2012, 2020):
    
    top_industries[year]={}
    dataset=pd.DataFrame()
      
    #order of groups on chart
    
    for s in range(len(stages)):
        top_industries[year][s]={'order': []}
        dataset=crunchbase_group.loc[year].loc[s].sort_values('usd_value')[::-1].iloc[:15]
        dataset['group']=pd.Series([industry_group(key) for key in dataset.index], index=dataset.index)
        
        for group in order:
            industries=[]
            dataset_group=dataset[dataset['group']==group].sort_values('usd_value')[::-1]
            for i, key in enumerate(list(dataset_group.index)):
                
                try: 
                    companies=get_companies(key, year, s)
                except:
                    companies=[]
                
                industries.append({'stage':s, 'name':key, 'group': industry_group(key), 
                                   'companies': companies,
                                   'value':np.round(np.float(dataset.loc[key]['usd_value']), 1), 
                                   'radius': np.round(np.float(np.sqrt(dataset.loc[key]['usd_value'])), 1)})
                
            top_industries[year][s]['order'].append({'group':group, 'industry':industries})

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [42]:
with open(str('top_industries.json'), 'w') as outfile:
        json.dump(top_industries, outfile)